In [31]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

In [32]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [33]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [34]:
model_df = df.copy()

model_df = model_df[model_df['artist'] == 'Chance the Rapper']

In [35]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [36]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
316,Childish Gambino,They Don't Like Me,[Verse 1: Chance The Rapper],"Eddy Scissorhands is workin' on my linin', Get...",Chance the Rapper,"[eddy, scissorhands, is, workin', on, my, lini..."
793,Childish Gambino,The Worst Guys (Live at Governor’s Ball),[Verse 2: Chance The Rapper],"You are the prototype, I love you like you lov...",Chance the Rapper,"[you, are, the, prototype, i, love, you, like,..."
824,Chance The Rapper,Favorite Song,[Verse 1: Chance The Rapper],"Chance, acid rapper, soccer, hacky sacker, Coc...",Chance the Rapper,"[chance, acid, rapper, soccer, hacky, sacker, ..."
825,Chance The Rapper,Favorite Song,[Chorus: Chance The Rapper],"This shit my favorite song, you just don't kno...",Chance the Rapper,"[this, shit, my, favorite, song, you, just, do..."
826,Chance The Rapper,Favorite Song,[Verse 2: Chance The Rapper],Young Rascal Flatts - young ass kid ass could ...,Chance the Rapper,"[young, rascal, flatts, -, young, ass, kid, as..."


In [37]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [38]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [39]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

316    eddy scissorhands is workin on my linin gettin...
793    you are the prototype i love you like you love...
824    chance acid rapper soccer hacky sacker cocky k...
825    this shit my favorite song you just dont know ...
826    young rascal flatts young ass kid ass could ra...
Name: verse, dtype: object

In [40]:
verses = list(model_df.verse.values)

In [41]:
verses[:10]

['eddy scissorhands is workin on my linin gettin spiffy for these bitches i be eyeing fredward mercury was playin while im ridin down the ryan in a scion and im flyin but its cool i got insurance on the run because i got warrants but its cool i got endurance in school i got suspended but in turn i got some earnings couldve spent it all on thursday but i saved it for my parents but i spent a bit on friday in the night i hang with donald im tired of mcdonalds want a chick that cook chicano that cant speak a lick of english and pronounce my name like chano she say something something spanish i look back like bitch i know tonight imma make decisions for life take some lucys boomers dark liq and light some might drive thats that shit right there that i dont like there that shit just had to say shouts to the niggas that passed away and shots for the niggas thats here right now and shots at the niggas that got away so sad to say but its all good niggas gon celebrate and wait for the day and w

In [42]:
all_verses_all_rappers = ','.join(verses)

all_verses_all_rappers[:100]

'eddy scissorhands is workin on my linin gettin spiffy for these bitches i be eyeing fredward mercury'

In [43]:
file = all_verses_all_rappers

In [44]:
# Read in text and change unicode characters to ASCII
import unidecode
import string
import random
import re

all_characters = string.printable
n_characters = len(all_characters)

file_len = len(file)
print(f'file_len = {file_len}')

file_len = 186806


In [45]:
chunk_len = 400

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

anding still i wont be moved how used to do when,i was a rock i was a rock and roller back in my day back in my day but now im just a rock i was a rock i was a rock i was a rock and roller back in my day back in my day but now im just a rock i was a rock i was a rock i was a rock and roller back in my day back in my day but now im just a rock i was a rock i was a rock and roller back in my day back


In [46]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        output = self.encoder(input.view(1, -1))
        output, hidden = self.gru(output.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        
        return output, hidden
    
    def init_hidden(self):
        return Variable(torch.randn(self.n_layers, 1, self.hidden_size))

In [47]:
def char2tensor(string):
    tensor = torch.zeros(len(string)).long()
    for char in range(len(string)):
        if string[char] in all_characters:
            tensor[char] = all_characters.index(string[char])
        else:
            tensor[char] = 94 #predict space if character unknown
        
    return Variable(tensor)

# Let's see it in action.
print(char2tensor('Metis0123abczABC'))

tensor([48, 14, 29, 18, 28,  0,  1,  2,  3, 10, 11, 12, 35, 36, 37, 38])


In [48]:
def random_training_set():
    chunk = random_chunk()
    inp = char2tensor(chunk[:-1])
    target = char2tensor(chunk[1:])
    return inp, target

In [49]:
def evaluate(model, prime_str='A', predict_len=100, temperature=0.8):
    
    hidden = model.init_hidden()
    prime_input = char2tensor(prime_str)
    predicted = prime_str
    
    # use priming string to build up hidden state
    
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char2tensor(predicted_char)
        
    return predicted

In [50]:
# helper function
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s/60)
    s -= m*60
    return '%dm %ds' % (m, s)

In [51]:
# The actual training part
def train(inp, target):
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0

    for char in range(chunk_len):
        output, hidden = model(inp[char], hidden)
        loss += criterion(output, target[char].unsqueeze(0))

    loss.backward()
    model_optimizer.step()

    return loss.data.item() / chunk_len

In [52]:
# # parameters
# n_epochs = 1000
# print_every = 100
# plot_every = 10
# hidden_size = 256
# n_layers = 2
# learning_rate = 0.001

# # model declaration
# model = RNN(n_characters, hidden_size, n_characters, n_layers)
# model_optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()

# start = time.time()
# all_losses = []
# loss_avg = 0

# for epoch in range(1, n_epochs + 1):
#     loss = train(*random_training_set())       
#     loss_avg += loss

#     if epoch % print_every == 0:
#         print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
#         print(evaluate(model, 'A ', 100), '\n')

#     if epoch % plot_every == 0:
#         all_losses.append(loss_avg / plot_every)
#         loss_avg = 0

In [53]:
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# %matplotlib inline

# plt.figure()
# plt.plot(all_losses)

In [54]:
# with open('models/chance_rnn_generator.pkl', 'wb') as picklefile:
#     pickle.dump(model, picklefile)

In [55]:
# Evaluate model from this notebook
print(evaluate(model, 'To be or not to be: ', predict_len=200, temperature=0.5))

To be or not to be: a love him the praise a good the same drugs good i fucked no bout wanna the kids even in the ready the better to the way the all your more a dream i dont do the gotta same a cresse i come and we dont 


In [56]:
# Evaluate model from this notebook
print(evaluate(model, 'I miss you girl come back ', predict_len=200, temperature=0.5))

I miss you girl come back i dont on the passings to the ready save to me summer came the dont do the dont so work in the same we dont do the same you to stay and the same drugs dont do the all a cready a fucking with the care 


In [57]:
# Evaluate model from this notebook
print(evaluate(model, "Started from the bottom", predict_len=200, temperature=0.4))

Started from the bottom on the niggas of the same and you say the dont do the same drugs dont all we dont do what here the got the was the say dont do the day the praise you ready to the sade you got stay the for the same a


In [58]:
# Evaluate model from this notebook
print(evaluate(model, "I'm the realest rapper alive", predict_len=200, temperature=0.4))

I'm the realest rapper alive math i way no more the more to the same the same you and in the rain cells dont do the better with the same for the blessings on the same drugs on the fuck you good i dont the way the ready a call th


In [59]:
# Evaluate model from this notebook
print(evaluate(model, "Smoke a lot of weed", predict_len=200, temperature=0.4))

Smoke a lot of weed in the ready so the same did the same you could the same with of miss the got she got the dont do the same drugs no more we dont do the fam to the we dont do the dont say she dont to the dont need a 


In [60]:
# Evaluate model from this notebook
print(evaluate(model, "Baby come", predict_len=200, temperature=0.4))

Baby come for you scare and in the hate house a with you tone wait no do the dont do the same drugs to the am the same to the round the ready than i got we dont do the got the rapper to the more you same you a


In [61]:
prompts = ['I miss you girl come back ', 'To be or not to be: ', "Started from the bottom", "I'm the realest rapper alive",
           "Smoke a lot of weed", "Baby come", 'Date a scientist', 'Butter my biscuit', 'Scramble my ', 'I sleep ',
           'Green eggs and ', 'Rollin in my', 'Drop top', 'Math is fun', 'Sentences', 'Coffee in the', 'Table that',
           'Homies', 'From the streets', 'Maserati', 'Air force one', 'President', 'Trump']

chance_rnn_verses = []

for i in range(len(prompts)):
    chance_rnn_verses.append(evaluate(model, prompts[i], predict_len=200, temperature=0.5))
    
    
chance_rnn_verses

['I miss you girl come back i dont we dont do the fucked a i dont we dont need a same dont do the same the dont get so do the same the hate do the same my niggas a call of the back we dont do the curse with the so got no birth h',
 'To be or not to be: it do the same drugs so mama ring to i aint wend i wape to the ready on the back do the say no bent you same a christ you care to the be dont do the same drugs dont a dont do the part ready i say dont',
 'Started from the bottome the had funny way i dont do the do the same than the got the ass and i got more i dont do the same your for i more and you really say it dont do the same drug my for you talk out i all the ready on ',
 "I'm the realest rapper alive with you and i got i burned in the ready to smoke and the ready same a care to the dont do the same drugs to the gon see of the grown ring ready with your pet to so we dont do the we dont do the do t",
 'Smoke a lot of weed the praying a dont do the ring a burned to the care take you 

In [62]:
columns = ['generated_verses']

chance_generated_df = pd.DataFrame(chance_rnn_verses, columns=columns)

In [63]:
chance_generated_df['artist'] = 'Chance the Rapper'

chance_generated_df

,generated_verses,artist
0,I miss you girl come back i dont we dont do th...,Chance the Rapper
1,To be or not to be: it do the same drugs so ma...,Chance the Rapper
2,Started from the bottome the had funny way i d...,Chance the Rapper
3,I'm the realest rapper alive with you and i go...,Chance the Rapper
4,Smoke a lot of weed the praying a dont do the ...,Chance the Rapper
5,Baby come i need lice and a but the feel a lik...,Chance the Rapper
6,Date a scientist on the same what the for my f...,Chance the Rapper
7,Butter my biscuit may and fore i dont do the r...,Chance the Rapper
8,Scramble my call the was a burned summer i wan...,Chance the Rapper
9,I sleep in the way i can a and im on the more ...,Chance the Rapper


In [64]:
with open('data/chance_generated_raps.pkl', 'wb') as picklefile:
    pickle.dump(chance_generated_df, picklefile)